In [ ]:
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from datasets import load_metric

In [ ]:
import pandas as pd
from datasets import Dataset

# Load your data from a CSV file
# Replace 'your_data.csv' with the actual file path to your CSV file
data = pd.read_csv('/Users/saivinilreddy/Downloads/SMA-Q1 &Q2/test.csv')

# Create a dataset from the CSV data
dataset = Dataset.from_pandas(data)

In [ ]:
import csv
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the pre-trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

# Function to generate outputs for long input text by dividing it into chunks
def generate_output(input_text):
    max_chunk_length = 500
    input_chunks = [input_text[i:i + max_chunk_length] for i in range(0, len(input_text), max_chunk_length)]
    output_chunks = []

    for chunk in input_chunks:
        inputs = tokenizer(chunk, return_tensors="pt")
        outputs = model.generate(**inputs, max_length=10000)
        decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        output_chunks.append(decoded_output[0])

    return "".join(output_chunks)

# Read input from CSV and write outputs to CSV in chunks of 50 rows
input_csv_file = "test.csv"
output_csv_file = "output.csv"
batch_size = 5

with open(input_csv_file, "r", encoding="utf-8") as csvfile, open(output_csv_file, "w", newline="", encoding="utf-8") as csvfile_out:
    reader = csv.reader(csvfile)
    writer = csv.writer(csvfile_out)

    # Write header to output CSV file if needed
    # writer.writerow(["Output"])

    input_batch = []
    for idx, row in enumerate(reader, 1):
        input_text = row[0]  # Assuming the input is in the first column of the CSV
        input_batch.append(input_text)

        # Process batch of 50 rows at a time
        if idx % batch_size == 0 or idx == batch_size:
            output_batch = [generate_output(text) for text in input_batch]
            for output_text in output_batch:
                writer.writerow([output_text])
            input_batch = []

print("Outputs generated and saved to output.csv.")

In [ ]:
pip install evaluate

In [2]:
import pandas as pd
from evaluate import load

# Load the BERT score model
bertscore = load("bertscore")

# Read data from CSV file
df = pd.read_csv("final.csv")

# Extract actual and predicted values from the DataFrame
predictions = df["Predicted"].tolist()
references = df["Actual"].tolist()

# Compute BERT scores for each pair of actual and predicted values
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

# Extract individual BERT scores (precision, recall, F1) for each pair
precision = results["precision"]
recall = results["recall"]
f1 = results["f1"]

# Create a new DataFrame to store the BERT scores
output_df = pd.DataFrame({"Actual": references, "Predicted": predictions, "Precision": precision, "Recall": recall, "F1": f1})

# Save the BERT scores to a new CSV file
output_df.to_csv("bert_scores_output.csv", index=False)

# Calculate the final cumulative BERT scores
final_precision = sum(precision) / len(precision)
final_recall = sum(recall) / len(recall)
final_f1 = sum(f1) / len(f1)

# Print the final cumulative BERT scores
print("Final Cumulative BERT Scores:")
print("Precision:", final_precision)
print("Recall:", final_recall)
print("F1 Score:", final_f1)

Final Cumulative BERT Scores:
Precision: 0.7750833320617676
Recall: 0.7775522385835647
F1 Score: 0.7750406132936478


In [3]:
import pandas as pd
from evaluate import load

# Load the ROUGE score model
rouge = load('rouge')

# Read data from CSV file
df = pd.read_csv("final.csv")

# Extract actual and predicted values from the DataFrame
predictions = df["Predicted"].tolist()
references = df["Actual"].tolist()

# Compute ROUGE scores for each pair of actual and predicted values
results = rouge.compute(predictions=predictions, references=references)

# Save the ROUGE scores to a new CSV file
output_df = pd.DataFrame({"Actual": references, "Predicted": predictions, "ROUGE-1": results['rouge1'], "ROUGE-2": results['rouge2'], "ROUGE-L": results['rougeL'], "ROUGE-Lsum": results['rougeLsum']})

output_df.to_csv("rouge_scores_output.csv", index=False)

# Print the ROUGE scores
print("ROUGE Scores:")
print("ROUGE-1:", results['rouge1'])
print("ROUGE-2:", results['rouge2'])
print("ROUGE-L:", results['rougeL'])
print("ROUGE-Lsum:", results['rougeLsum'])

ROUGE Scores:
ROUGE-1: 0.2690847700213629
ROUGE-2: 0.09000883228910794
ROUGE-L: 0.1785974257179503
ROUGE-Lsum: 0.17822636563720617


In [5]:
import pandas as pd
from evaluate import load
from nltk.translate.bleu_score import sentence_bleu

# Load BLEU score function from NLTK
def compute_bleu_score(prediction, references):
    return sentence_bleu(references, prediction)

# Read data from CSV file
df = pd.read_csv("final.csv")

# Extract actual and predicted values from the DataFrame
predictions = df["Predicted"].tolist()
references_list = df["Actual"].apply(lambda x: [reference.strip() for reference in x.split(',')]).tolist()

# Compute BLEU scores for each pair of actual and predicted values
bleu_scores = [compute_bleu_score(prediction, references) for prediction, references in zip(predictions, references_list)]

# Create a new DataFrame to store the BLEU scores
output_df = pd.DataFrame({"Actual": df["Actual"], "Predicted": predictions, "BLEU Score": bleu_scores})

# Save the BLEU scores to a new CSV file
output_df.to_csv("bleu_scores_output.csv", index=False)

# Calculate the average BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores)

# Print the average BLEU score
print("Average BLEU Score:", average_bleu_score)

/Users/saivinilreddy/anaconda3/envs/GPU/lib/python3.11/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Average BLEU Score: 0.29171001991417306
